<a href="https://colab.research.google.com/github/Jarvis-BITS/midodrine-mimic-iv/blob/demo/notebook/mice_imputing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
!pip install rpy2

In [7]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("mice")

### Data Imputation for tmle

---
Dashti, Ghazaleh, and Lee, Katherine J, et al. "Handling missing data for causal effect estimation in cohort studies using Targeted Maximum Likelihood Estimation". INTERNATIONAL JOURNAL OF EPIDEMIOLOGY, vol.50, 2021, pp. 55-55. doi:10.1093/ije/dyab168.150

In [ ]:
%%R
library(mice)

In [10]:
%%R
full_data <- read.csv("https://raw.githubusercontent.com/Jarvis-BITS/midodrine-mimic-iv/demo/data/table_cohort.csv")

In [ ]:
%%R
class(full_data)

[1] "data.frame"


In [ ]:
%%R
as_tibble(full_data)

# A tibble: 35,300 × 50
    stay_id hadm_id subject_id   age Gender ethnicity midodrine avg_doses_per_2…
      <int>   <int>      <int> <int>  <int> <chr>         <int>            <dbl>
 1 36014877  2.72e7   11861017    87      0 WHITE             1             1.60
 2 32272859  2.60e7   11259141    60      1 WHITE             1             1.63
 3 34023828  2.18e7   18010960    59      0 UNKNOWN           1             1.59
 4 37798199  2.98e7   10926537    42      1 BLACK/AF…         1             1.57
 5 32309766  2.93e7   15703353    59      1 WHITE             1             1.52
 6 32470068  2.99e7   12682108    32      1 WHITE             1             1.67
 7 39913468  2.56e7   14695267    26      0 UNKNOWN           1             1.96
 8 36158255  2.69e7   11544355    29      0 UNKNOWN           1             1.69
 9 39406131  2.71e7   16583373    51      0 WHITE             1             1.64
10 38515563  2.42e7   11896370    62      1 WHITE             1             1.37
# … 

In [11]:
%%R
features = subset(full_data,select=-c(stay_id,hadm_id,subject_id,ethnicity,midodrine,avg_doses_per_24_hrs,mortality,mort_28_day,vasopressor_use))

In [ ]:
%%R
str(features)

'data.frame':	35300 obs. of  41 variables:
 $ age              : int  87 60 59 42 59 32 26 29 51 62 ...
 $ Gender           : int  0 1 0 1 1 1 0 0 0 1 ...
 $ MICU             : int  0 0 0 0 0 0 0 0 1 0 ...
 $ adm_weekday      : int  1 7 1 7 6 3 2 7 1 5 ...
 $ weight           : num  70 81.6 90 73 72.7 86.3 81 66.8 62 111 ...
 $ sapsii           : num  42 34 15 30 20 18 38 24 31 23 ...
 $ charlson_index   : num  8.83 3.33 3 2 3 ...
 $ CPD_or_COPD_rate : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Maligancy_rate   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ mild_liver_rate  : num  0 0 0 0 0 0 0 0 1 0 ...
 $ severe_liver_rate: num  0 0 0 0 0 0 0 0 0 0 ...
 $ Renal_rate       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ CAD_rate         : num  1 0 0 0 0 0 0 0 0 0 ...
 $ AFIB_rate        : num  0 1 0 0 0 0 0 0 1 0 ...
 $ hemoglobin       : num  10.4 11.2 13.2 12.4 8 ...
 $ platelet         : num  273 238 157 195 223 ...
 $ wbc              : num  11.7 10.6 11.5 12.1 10.2 ...
 $ sodium           : num  136 135 142 140 136 .

In [12]:
%%R
imputed_data <- mice(features, method="cart", m=3,  maxit=5,seed=500)
features_complete <- complete(imputed_data) 


 iter imp variable
  1   1  weight  sapsii  hemoglobin  platelet  wbc  sodium  potassium  bicarbonate  chloride  bun  creatinine  lactate  ph  po2  pco2  heart_rate_mean  sbp_mean  dbp_mean  mbp_mean  resp_rate_mean  temperature_mean  spo2_mean  glucose_mean  sedative_use
  1   2  weight  sapsii  hemoglobin  platelet  wbc  sodium  potassium  bicarbonate  chloride  bun  creatinine  lactate  ph  po2  pco2  heart_rate_mean  sbp_mean  dbp_mean  mbp_mean  resp_rate_mean  temperature_mean  spo2_mean  glucose_mean  sedative_use
  1   3  weight  sapsii  hemoglobin  platelet  wbc  sodium  potassium  bicarbonate  chloride  bun  creatinine  lactate  ph  po2  pco2  heart_rate_mean  sbp_mean  dbp_mean  mbp_mean  resp_rate_mean  temperature_mean  spo2_mean  glucose_mean  sedative_use
  2   1  weight  sapsii  hemoglobin  platelet  wbc  sodium  potassium  bicarbonate  chloride  bun  creatinine  lactate  ph  po2  pco2  heart_rate_mean  sbp_mean  dbp_mean  mbp_mean  resp_rate_mean  temperature_mean  sp

In [14]:
%%R
summary(features_complete)

      age             Gender            MICU         adm_weekday   
 Min.   : 18.00   Min.   :0.0000   Min.   :0.0000   Min.   :1.000  
 1st Qu.: 51.00   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:2.000  
 Median : 64.00   Median :0.0000   Median :1.0000   Median :4.000  
 Mean   : 62.71   Mean   :0.4698   Mean   :0.5343   Mean   :3.993  
 3rd Qu.: 77.00   3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:6.000  
 Max.   :102.00   Max.   :1.0000   Max.   :1.0000   Max.   :7.000  
                                                                   
     weight           sapsii       charlson_index   CPD_or_COPD_rate
 Min.   :   1.0   Min.   :  0.00   Min.   : 0.000   Min.   :0.0000  
 1st Qu.:  64.7   1st Qu.: 24.00   1st Qu.: 3.000   1st Qu.:0.0000  
 Median :  77.1   Median : 32.00   Median : 5.000   Median :0.0000  
 Mean   :  80.7   Mean   : 34.06   Mean   : 5.026   Mean   :0.2337  
 3rd Qu.:  92.4   3rd Qu.: 42.00   3rd Qu.: 7.000   3rd Qu.:0.0000  
 Max.   :1120.0   Max.   :114.00   Max.   

In [15]:
%%R
summary(features)

      age             Gender            MICU         adm_weekday   
 Min.   : 18.00   Min.   :0.0000   Min.   :0.0000   Min.   :1.000  
 1st Qu.: 51.00   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:2.000  
 Median : 64.00   Median :0.0000   Median :1.0000   Median :4.000  
 Mean   : 62.71   Mean   :0.4698   Mean   :0.5343   Mean   :3.993  
 3rd Qu.: 77.00   3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:6.000  
 Max.   :102.00   Max.   :1.0000   Max.   :1.0000   Max.   :7.000  
                                                                   
     weight            sapsii       charlson_index   CPD_or_COPD_rate
 Min.   :   1.00   Min.   :  0.00   Min.   : 0.000   Min.   :0.0000  
 1st Qu.:  64.70   1st Qu.: 25.00   1st Qu.: 3.000   1st Qu.:0.0000  
 Median :  77.10   Median : 33.00   Median : 5.000   Median :0.0000  
 Mean   :  80.65   Mean   : 34.97   Mean   : 5.026   Mean   :0.2337  
 3rd Qu.:  92.30   3rd Qu.: 43.00   3rd Qu.: 7.000   3rd Qu.:0.0000  
 Max.   :1120.00   Max.   :114.00   

In [17]:
%%R
write.csv(features_complete, file='features_complete.csv')